In [ ]:
# !pip install requests beautifulsoup4
# !pip install tqdm

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import tqdm

url = "https://www.serpom.org.ua/fuckups/page/"
pages_count = [x for x in range(10)]

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

In [3]:
def get_member(member_url):
    member_response = requests.get(member_url, headers=headers)
    if member_response.status_code == 200:
        member_soup = BeautifulSoup(member_response.content, "html.parser")
        member_name = member_soup.find('h1', class_='deputy-card-single__title').get_text(strip=True)
        fuckups_count = member_soup.find('div', class_='count-fuckups').get_text(strip=True)
        deputy_card = member_soup.find('span', class_='deputy-card-single-info__title').get_text(strip=True)
        return member_name, fuckups_count, deputy_card

def get_members_info(members_url):
    all_members = []
    member_response = requests.get(members_url, headers=headers)
    if member_response.status_code == 200:
        member_soup = BeautifulSoup(member_response.content, "html.parser")
        member_names = member_soup.find_all('h3', class_='deputy-card__name')
        for member in member_names:
            title_a = member.find('a')
            title_ref = title_a.get('href')
            member_name, fuckups_count, deputy_card = get_member(title_ref)
            all_members.append((member_name, fuckups_count, deputy_card))
        return all_members

    else:
        print(f"Failed to retrieve member page. Status code: {member_response.status_code}")
        return None, None, None

In [ ]:
fuckups = []
all_deputies = {}

for page in pages_count:
    print(page)
    
    new_url = url + str(page) +"/"
    response = requests.get(new_url, headers=headers)


    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        
        for entry in soup.find_all('div', class_='article-card-container'):
            title = entry.find('h3', class_='article-card__title')
            
            title_text = title.get_text(strip=True)

            title_a = title.find('a')
            title_ref = title_a.get('href')

            all_members_info = get_members_info(title_ref)
            for name, count, party in all_members_info:
                if name not in all_deputies.keys():
                    all_deputies[name] = (count, party)

            all_members_names_list = [x[0] for x in all_members_info]
            
            fuckups.append({'title': title_text, 'members': all_members_names_list})
            print(all_members_names_list)
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

for item in fuckups:
    print(f"Title: {item['title']}, Members: {', '.join(item['members'])}")


In [19]:
all_deputies

{'Войцехівський Віталій Олександрович': ['4', 'Слуга Народу'],
 'Ковальчук Олександр Володимирович': ['18', 'Слуга Народу'],
 'Бужанський Максим Аркадійович': ['31', 'Слуга Народу'],
 'Бабенко Микола Вікторович': ['18', 'Довіра'],
 'Бондар Ганна Вячеславівна': ['8', 'Слуга Народу'],
 'Кіссе Антон Іванович': ['49', 'Довіра'],
 'Галушко Микола Леонідович': ['18', 'Слуга Народу'],
 'Дануца Олександр Анатолійович': ['20', 'Слуга Народу'],
 'Касай Геннадій Олександрович': ['17', 'Слуга Народу'],
 'Борзова Ірина Наумівна': ['18', 'Слуга Народу'],
 'Дмитрук Артем Геннадійович': ['19', 'Позафракційні'],
 'Лунченко Валерій Валерійович': ['35', 'Довіра'],
 'Каптєлов Роман Володимирович': ['22', 'Слуга Народу'],
 'Литвиненко Сергій Анатолійович': ['26', 'Слуга Народу'],
 'Іонушас Сергій Костянтинович': ['20', 'Слуга Народу'],
 'Іоффе Юлій Якович': ['43', 'ЕксОПЗЖ'],
 'Бойко Юрій Анатолійович': ['23', 'ЕксОПЗЖ'],
 'Борт Віталій Петрович': ['24', 'ЕксОПЗЖ'],
 'Брагар Євгеній Вадимович': ['22', 'Слу

In [15]:
# with open('all_deputies.json', 'w', encoding='utf-8') as file:
#     json.dump(all_deputies, file, ensure_ascii=False, indent=4)

with open('all_deputies.json', 'r', encoding='utf-8') as file:
    all_deputies = json.load(file)

Dictionary saved as 'all_deputies.json'


In [20]:
# with open('fuckups_data.json', 'w', encoding='utf-8') as f:
#     json.dump(fuckups, f, ensure_ascii=False, indent=4)

with open('fuckups_data.json', 'r', encoding='utf-8') as file:
    fuckups = json.load(file)

Data saved to 'fuckups_data_updated.json'


In [34]:
friends_of_deputy = {}
strongly_voted = {}
never_voted = {}

for deputy, features in tqdm.tqdm(all_deputies.items()):   
    print(deputy) 
    
    name_splited = deputy.split(" ")
    if len(name_splited) == 3:
        sur, name, _ = name_splited
    elif len(name_splited) == 2:
        sur, name = name_splited
    else:
        break
    try:
        link_for_search = f"https://rada4you.org/search?query={name}+{sur}&button=hero_search"
        response = requests.get(link_for_search, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            ress = soup.find('h1', class_='media-heading')
            dep_found = ress.get_text(strip=True)

            table = soup.find('table', class_="table")
            res = table.find_all("tr")
            friends = []

            for x in res[1:]:
                # print(x.get_text().split("\n"))
                _, perc, friend_ped_name, city, party, _ = x.get_text().split("\n")
                perc = int(perc.replace("%", ""))
                friends.append((perc, friend_ped_name, party))
                
                # print(perc, friend_ped_name, party)
                # print("-")
                # break
            friends_of_deputy[deputy] = friends

            never_voted_on = soup.find('div', class_='position-has-never-voted-on')
            poss = never_voted_on.find_all("li", class_ = "object-item")
            all_negatives = []
            for pos in poss:
                all_negatives.append(pos.get_text(strip=True))
            never_voted[deputy] = all_negatives

            strongly_voted_on = soup.find('div', class_='position-voted-very-strongly-for')
            poss = strongly_voted_on.find_all("li", class_ = "object-item")
            all_strong = []
            for pos in poss:
                all_strong.append(pos.get_text(strip=True))
            strongly_voted[deputy] = all_strong

    except:
        print("error with:", link_for_search)

  0%|          | 0/104 [00:00<?, ?it/s]

Войцехівський Віталій Олександрович


  1%|          | 1/104 [00:03<05:43,  3.34s/it]

Ковальчук Олександр Володимирович


  2%|▏         | 2/104 [00:23<22:57, 13.50s/it]

...

 98%|█████████▊| 102/104 [04:40<00:03,  1.61s/it]

Приходько Наталія Ігорівна


 99%|█████████▉| 103/104 [04:42<00:01,  1.83s/it]

Бондар Віктор Васильович


100%|██████████| 104/104 [04:45<00:00,  2.74s/it]


In [27]:
with open('friends_of_deputy.json', 'w', encoding='utf-8') as f:
    json.dump(friends_of_deputy, f, ensure_ascii=False, indent=4)

with open('strongly_voted.json', 'w', encoding='utf-8') as f:
    json.dump(strongly_voted, f, ensure_ascii=False, indent=4)

with open('never_voted.json', 'w', encoding='utf-8') as f:
    json.dump(never_voted, f, ensure_ascii=False, indent=4)

Data saved to 'friends_of_deputy.json'


# Neo4j

In [ ]:
# !pip install neo4j

In [ ]:
from neo4j import GraphDatabase
import json

In [ ]:
json_file_path = 'fuckups_data.json'

uri = "bolt://localhost:7689"
username = "neo4j" 
password = "12345678"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [80]:
def create_fuckup_data(tx, title, members, all_deputies):    
    for member in members:
        tx.run("""
            MERGE (t:Title {name: $title})
            MERGE (m:Member {name: $member, number_of_fuckups: $number_of_fuckups})
            MERGE (m)-[:SUPPORTS]->(t)
        """, member=member, title=title, number_of_fuckups=all_deputies[member][0])

    for member in members:
        tx.run("""
            MERGE (t:Party {name: $party})
            MERGE (m:Member {name: $member})
            MERGE (m)-[:member_of_the_party]->(t)
        """, member=member, party=all_deputies[member][1])


def votes_for(tx, votes):
    for dep, vote in votes.items():
        tx.run("""
            MERGE (t:Vote {name: $title})
            MERGE (m:Member {name: $member})
            MERGE (m)-[:always_vote_for]->(t)
        """, member=dep, title=vote)

def votes_against(tx, votes):
    for dep, vote in votes.items():
        tx.run("""
            MERGE (t:Vote {name: $title})
            MERGE (m:Member {name: $member})
            MERGE (m)-[:always_vote_against]->(t)
        """, member=dep, title=vote)

def create_friends_relation(tx, friends_of_deputy, related_names):
    for dep, friends_data in friends_of_deputy.items():
        for friend_data in friends_data:
            if friend_data[1] in related_names.keys():
                tx.run("""
                    MERGE (friend:Member {name: $friend})
                    MERGE (m:Member {name: $member})
                    MERGE (m)-[:votes_similar {similarity: $perc}]->(friend)
                """, member=dep, friend=related_names[friend_data[1]], perc=friend_data[0])


def load_data_to_neo4j(data, all_deputies, strongly_voted, never_voted, friends_of_deputy, related_names):
    
    with driver.session() as session:
        for item in fuckups:
            title = item['title']
            members = item['members']
            session.write_transaction(create_fuckup_data, title, members, all_deputies)
        session.write_transaction(votes_for, strongly_voted)
        session.write_transaction(votes_against, never_voted)
        # session.write_transaction(create_friends_relation, friends_of_deputy, related_names)
        

    print("Data successfully loaded to Neo4j!")

In [ ]:
def calculate_similarity_and_create_relations(tx):
    tx.run("""
    CALL gds.graph.project.cypher(
    'memberGraphMain',
    'MATCH (m:Member) RETURN id(m) AS id',
    'MATCH (m1:Member)-[:SUPPORTS]->(v:Title)<-[:SUPPORTS]-(m2:Member)
     RETURN id(m1) AS source, id(m2) AS target
     UNION
     MATCH (m1:Member)-[:always_vote_for]->(t:Vote)<-[:always_vote_for]-(m2:Member)
     RETURN id(m1) AS source, id(m2) AS target'
)
    """)

    similarity_results = tx.run("""
    CALL gds.nodeSimilarity.stream('memberGraphMain1')
    YIELD node1, node2, similarity
    RETURN gds.util.asNode(node1).name AS member1, 
        gds.util.asNode(node2).name AS member2, 
        similarity
    ORDER BY similarity DESC

    """)

    for record in similarity_results:
        tx.run("""
            MATCH (m1:Member {name: $member1})
            MATCH (m2:Member {name: $member2})
            MERGE (m1)-[:similar_to {similarity_perc: $similarity}]->(m2)
        """, member1=record["member1"], member2=record["member2"], similarity=record["similarity"] * 100)

    tx.run("CALL gds.graph.drop('memberGraph')")

In [65]:
related_names = {}

for _, friends_data in friends_of_deputy.items():
    for friend_data in friends_data:
        friend = friend_data[1]
        for deputy in all_deputies.keys():
            related_name = ""
            name_splited = deputy.split(" ")
            if len(name_splited) == 3:
                sur, name, _ = name_splited
                if name + " " + sur == friend:
                    related_name = deputy
            elif len(name_splited) == 2:
                sur, name = name_splited
                if name + " " + sur == friend:
                    related_name = deputy
            else:
                break
            if related_name:
                related_names[friend] = related_name

In [81]:
load_data_to_neo4j(fuckups, all_deputies, strongly_voted, never_voted, friends_of_deputy, related_names)

/var/folders/pm/44lz1vt16w53yrztk3stvdc40000gn/T/ipykernel_5089/3742549946.py:50: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_fuckup_data, title, members, all_deputies)
/var/folders/pm/44lz1vt16w53yrztk3stvdc40000gn/T/ipykernel_5089/3742549946.py:51: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(votes_for, strongly_voted)
/var/folders/pm/44lz1vt16w53yrztk3stvdc40000gn/T/ipykernel_5089/3742549946.py:52: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(votes_against, never_voted)


Data successfully loaded to Neo4j!


In [ ]:
driver.close()